In [4]:
import pandas as pd


In [5]:
olist_orders= pd.read_csv("olist_orders_dataset.csv")
olist_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [6]:
olist_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [7]:
olist_orders["order_purchase_timestamp"]= pd.to_datetime(olist_orders["order_purchase_timestamp"]) 
#Hay que pasarlo a formato datatime o da error porque es un object

In [8]:
#Cálculo para definir el churn (6 meses sin realizar compras)

In [9]:
#Saco última fecha compra por cliente, que resulta que solo tenemos una fecha, pero bueno.
ultima_compra = olist_orders.groupby("customer_id")["order_purchase_timestamp"].max().reset_index()


In [10]:
#Veo cuántos días llevan sin comprar
fecha_referencia = pd.to_datetime('2018-10-17')  #Esta es la última fecha del dataset
días_sin_comprar = (fecha_referencia - ultima_compra["order_purchase_timestamp"]).dt.days 
#.dt.days se necesita para sacar días enteros

In [11]:
ultima_compra["dias_desde_ultima_compra"] = días_sin_comprar #añado columna en la que me pone los días sin comprar


In [12]:
#Defini churn en cada cliente en base a 180 días sin comrar para traabjar el scoring voy a necesitar que sea numérico
ultima_compra["churn"] = ultima_compra["dias_desde_ultima_compra"].apply(lambda x: 1 if x > 180 else 0)

In [13]:
ultima_compra

,customer_id,order_purchase_timestamp,dias_desde_ultima_compra,churn
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,336,1
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,457,1
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,595,1
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,426,1
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,197,1
...,...,...,...,...
99436,fffecc9f79fd8c764f843e9951b11341,2018-03-29 16:59:26,201,1
99437,fffeda5b6d849fbd39689bb92087f431,2018-05-22 13:36:02,147,0
99438,ffff42319e9b2d713724ae527742af25,2018-06-13 16:57:05,125,0
99439,ffffa3172527f765de70084a7e53aae8,2017-09-02 11:53:32,409,1


In [16]:
ultima_compra.to_pickle("ultima_compra.pkl")
#formato Pickle, que es el formato nativo de Python para almacenar objetos.
#Hago esto para llevármelo al notebook de confirmación de hipótesis

